In [1]:
%pylab inline
import gc
import os
import fpfs
import numpy as np
import imgSimutil
import galsim
import astropy.io.fits as pyfits
hscDir=os.path.join(os.environ['homeWrk'],'S16ACatalogs','S16AStandardV2')

Populating the interactive namespace from numpy and matplotlib


In [2]:
cd wrkDir/FPFS2/sim

/lustre/work/xiangchong.li/work/FPFS2/sim


In [25]:
rotArray

array([0.       , 0.5      , 0.25     , 0.75     , 0.125    , 0.375    ,
       0.625    , 0.875    , 0.0625   , 0.1875   , 0.3125   , 0.4375   ,
       0.5625   , 0.6875   , 0.8125   , 0.9375   , 0.03125  , 0.09375  ,
       0.15625  , 0.21875  , 0.28125  , 0.34375  , 0.40625  , 0.46875  ,
       0.53125  , 0.59375  , 0.65625  , 0.71875  , 0.78125  , 0.84375  ,
       0.90625  , 0.96875  , 0.015625 , 0.046875 , 0.078125 , 0.109375 ,
       0.140625 , 0.171875 , 0.203125 , 0.234375 , 0.265625 , 0.296875 ,
       0.328125 , 0.359375 , 0.390625 , 0.421875 , 0.453125 , 0.484375 ,
       0.515625 , 0.546875 , 0.578125 , 0.609375 , 0.640625 , 0.671875 ,
       0.703125 , 0.734375 , 0.765625 , 0.796875 , 0.828125 , 0.859375 ,
       0.890625 , 0.921875 , 0.953125 , 0.984375 , 0.0078125, 0.0234375,
       0.0390625, 0.0546875, 0.0703125, 0.0859375, 0.1015625, 0.1171875,
       0.1328125, 0.1484375, 0.1640625, 0.1796875, 0.1953125, 0.2109375,
       0.2265625, 0.2421875, 0.2578125, 0.2734375, 

In [13]:
psfFWHM='60'
nx      =   100
ny      =   100
ngrid  =   64

g2       =   0.00
def measureShear(psfImg,galImgAll):
    fpTask=fpfs.fpfsBase.fpfsTask(psfImg)
    # Read GAL image
    imgList=[galImgAll[j*64:(j+1)*64,i*64:(i+1)*64] for i in range(100) for j in range(100)]

    # Measure FPFS moments
    a=fpTask.measure(imgList)

    # Measure FPFS ellipticity, FPFS response
    # The weighting parameter
    C=100
    # Again, for noiseless galaxies, you do not need to set rev=True
    # to revise second-order noise bias
    b=fpfs.fpfsBase.fpfsM2E(a,C)
    # Estimate shear
    g_est=np.average(b['fpfs_e1'])/np.average(b['fpfs_RE'])
    return g_est

In [11]:
FWHM =   eval(psfFWHM)/100.
psfInt  =   galsim.Moffat(beta=3.5,fwhm=FWHM,trunc=FWHM*4.)
psfInt  =   psfInt.shear(e1=0.02,e2=-0.02)
scale   =   0.168
npoints=  20

ud        =   galsim.UniformDeviate(1)
for g1,pend  in   zip([-0.02,0.02],['g1-0000','g1-2222']):
    for radius in [0.07,0.15,0.20]:
        gal_image   =   galsim.ImageF(nx*ngrid,ny*ngrid,scale=scale)
        gal_image.setOrigin(0,0)
        for ix in range(100):
            for iy in range(100):
                igal   =   ix*100+iy
                b       =   galsim.BoundsI(ix*ngrid,(ix+1)*ngrid-1,iy*ngrid,(iy+1)*ngrid-1)
                if igal%4==0:
                    gal0     =   galsim.RandomKnots(half_light_radius=radius,npoints=npoints,flux=10.,rng=ud)
                sub_gal_image = gal_image[b]
                ang    =   igal%4*np.pi/4. * galsim.radians
                gal     =   gal0.rotate(ang)
                # Shear the galaxy
                gal     =   gal.shear(g1=g1,g2=g2)
                gal     =   galsim.Convolve([psfInt,gal])
                # Draw the galaxy image
                gal.drawImage(sub_gal_image)
                del gal
                gc.collect()
        outfname=os.path.join('small%d_psf60' %int(radius*10),'image-0-%s.fits' %pend)
        print(outfname)
        pyfits.writeto(outfname,gal_image.array,overwrite=True)
        del gal_image
        gc.collect()

small0_psf60/image-0-g1-0000.fits
small1_psf60/image-0-g1-0000.fits
small2_psf60/image-0-g1-0000.fits
small0_psf60/image-0-g1-2222.fits
small1_psf60/image-0-g1-2222.fits
small2_psf60/image-0-g1-2222.fits


In [20]:
radius=0.07
psfFname=   os.path.join('./star_psf%s/' %psfFWHM,'psf-%s.fits' %psfFWHM)
psfData   =   pyfits.getdata(psfFname)
npad       =   (ngrid-psfData.shape[0])//2
psfData2 =   np.pad(psfData,(npad+1,npad),mode='constant')
assert psfData2.shape[0]==ngrid

pend='g1-0000'
gal_all=pyfits.getdata(os.path.join('small%d_psf60' %int(radius*10),'image-1-%s.fits' %pend))
gm=measureShear(psfData2,gal_all)
print(gm)
pend='g1-2222'
gal_all=pyfits.getdata(os.path.join('small%d_psf60' %int(radius*10),'image-1-%s.fits' %pend))
gp=measureShear(psfData2,gal_all)
(gp-gm)/0.04-1

-0.019992226314277148


-0.000394620698446424